In [1]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 DAY 0 DETECTION")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("Mission: Find the setup day BEFORE runs start")
print("="*60)

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 DAY 0 DETECTION
Date: 2026-01-06 22:58
Mission: Find the setup day BEFORE runs start


In [2]:
# Use serial runners from Notebook 1, or scan fresh
SERIAL_RUNNERS = ['SIDU', 'ASTS', 'IONQ', 'NVTS', 'QBTS', 'RGTI', 'USAR', 'WOLF', 'RKLB', 'LUNR']

print(f"Analyzing Day 0 patterns for: {SERIAL_RUNNERS}")

Analyzing Day 0 patterns for: ['SIDU', 'ASTS', 'IONQ', 'NVTS', 'QBTS', 'RGTI', 'USAR', 'WOLF', 'RKLB', 'LUNR']


In [3]:
def find_all_day1s(ticker, lookback_months=6):
    """
    Find all Day 1's (first green day of a 3+ day run).
    Returns list of Day 1 dates with context.
    """
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_months * 30)
        
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if len(hist) < 20:
            return None
        
        # Calculate metrics
        hist['green'] = hist['Close'] > hist['Close'].shift(1)
        hist['pct_change'] = hist['Close'].pct_change() * 100
        hist['vol_ratio'] = hist['Volume'] / hist['Volume'].rolling(20).mean()
        hist['gap'] = (hist['Open'] - hist['Close'].shift(1)) / hist['Close'].shift(1) * 100
        
        # Find Day 1's (first day of 3+ green streak with 10%+ total gain)
        day1s = []
        i = 0
        
        while i < len(hist) - 3:
            # Check if this starts a 3+ day run
            if hist['green'].iloc[i]:
                run_length = 1
                j = i + 1
                
                while j < len(hist) and hist['green'].iloc[j]:
                    run_length += 1
                    j += 1
                
                if run_length >= 3:
                    # Calculate total gain
                    start_price = hist['Close'].iloc[i-1] if i > 0 else hist['Open'].iloc[i]
                    end_price = hist['Close'].iloc[i + run_length - 1]
                    total_gain = ((end_price / start_price) - 1) * 100
                    
                    if total_gain >= 10:
                        day1s.append({
                            'date': hist.index[i],
                            'day1_idx': i,
                            'run_length': run_length,
                            'total_gain': total_gain,
                            'day1_gain': hist['pct_change'].iloc[i],
                            'day1_vol_ratio': hist['vol_ratio'].iloc[i],
                            'day1_gap': hist['gap'].iloc[i]
                        })
                    
                    i = j  # Skip to end of run
                    continue
            
            i += 1
        
        return day1s, hist
        
    except Exception as e:
        print(f"Error for {ticker}: {e}")
        return None, None

print("✅ find_all_day1s() function ready")

✅ find_all_day1s() function ready


In [4]:
def analyze_day0(day1_info, hist):
    """
    Analyze what happened on Day 0 (day before Day 1).
    Also looks at Day -1, Day -2 for patterns.
    """
    idx = day1_info['day1_idx']
    
    # Need at least 3 days of history before Day 1
    if idx < 3:
        return None
    
    day0_analysis = {
        'day1_date': day1_info['date'],
        'run_length': day1_info['run_length'],
        'total_gain': day1_info['total_gain']
    }
    
    # Day 0 (day before Day 1)
    day0_analysis['day0'] = {
        'date': hist.index[idx - 1],
        'green': hist['green'].iloc[idx - 1],
        'pct_change': hist['pct_change'].iloc[idx - 1],
        'vol_ratio': hist['vol_ratio'].iloc[idx - 1],
        'close': hist['Close'].iloc[idx - 1],
        'high': hist['High'].iloc[idx - 1],
        'low': hist['Low'].iloc[idx - 1]
    }
    
    # Day -1 (2 days before Day 1)
    day0_analysis['day_m1'] = {
        'date': hist.index[idx - 2],
        'green': hist['green'].iloc[idx - 2],
        'pct_change': hist['pct_change'].iloc[idx - 2],
        'vol_ratio': hist['vol_ratio'].iloc[idx - 2]
    }
    
    # Day -2 (3 days before Day 1)
    day0_analysis['day_m2'] = {
        'date': hist.index[idx - 3],
        'green': hist['green'].iloc[idx - 3],
        'pct_change': hist['pct_change'].iloc[idx - 3],
        'vol_ratio': hist['vol_ratio'].iloc[idx - 3]
    }
    
    # Day 1 details
    day0_analysis['day1'] = {
        'date': hist.index[idx],
        'pct_change': day1_info['day1_gain'],
        'vol_ratio': day1_info['day1_vol_ratio'],
        'gap': day1_info['day1_gap']
    }
    
    return day0_analysis

print("✅ analyze_day0() function ready")

✅ analyze_day0() function ready


In [5]:
# COLLECT ALL DAY 0 DATA
print("\n" + "="*60)
print("🔍 COLLECTING DAY 0 DATA FROM ALL RUNS")
print("="*60)

all_day0_data = []

for ticker in SERIAL_RUNNERS:
    print(f"\n📊 {ticker}:")
    
    day1s, hist = find_all_day1s(ticker)
    
    if day1s is None:
        print(f"   ❌ No data")
        continue
    
    print(f"   Found {len(day1s)} Day 1's in last 6 months")
    
    for day1 in day1s:
        day0 = analyze_day0(day1, hist)
        if day0:
            day0['ticker'] = ticker
            all_day0_data.append(day0)

print(f"\n\nTotal Day 0 samples collected: {len(all_day0_data)}")


🔍 COLLECTING DAY 0 DATA FROM ALL RUNS

📊 SIDU:
   Found 7 Day 1's in last 6 months

📊 ASTS:
   Found 10 Day 1's in last 6 months

📊 IONQ:
   Found 8 Day 1's in last 6 months

📊 NVTS:
   Found 6 Day 1's in last 6 months

📊 QBTS:
   Found 9 Day 1's in last 6 months

📊 RGTI:
   Found 10 Day 1's in last 6 months

📊 USAR:
   Found 5 Day 1's in last 6 months

📊 WOLF:
   Found 2 Day 1's in last 6 months

📊 RKLB:
   Found 8 Day 1's in last 6 months

📊 LUNR:
   Found 8 Day 1's in last 6 months


Total Day 0 samples collected: 68


In [6]:
# ANALYZE DAY 0 PATTERNS
if all_day0_data:
    print("\n" + "="*80)
    print("📊 DAY 0 PATTERN ANALYSIS")
    print("="*80)
    
    # Convert to analyzable format
    day0_df = pd.DataFrame([
        {
            'ticker': d['ticker'],
            'day0_green': d['day0']['green'],
            'day0_pct': d['day0']['pct_change'],
            'day0_vol': d['day0']['vol_ratio'],
            'day_m1_green': d['day_m1']['green'],
            'day_m1_pct': d['day_m1']['pct_change'],
            'day_m2_green': d['day_m2']['green'],
            'day1_pct': d['day1']['pct_change'],
            'day1_vol': d['day1']['vol_ratio'],
            'day1_gap': d['day1']['gap'],
            'run_length': d['run_length'],
            'total_gain': d['total_gain']
        }
        for d in all_day0_data
    ])
    
    print(f"\nSample size: {len(day0_df)} Day 1's analyzed\n")
    
    # PATTERN 1: Day 0 Color
    print("\n📈 PATTERN 1: Day 0 Color (Green vs Red before Day 1)")
    print("-" * 50)
    green_pct = day0_df['day0_green'].sum() / len(day0_df) * 100
    red_pct = 100 - green_pct
    print(f"   Day 0 was GREEN: {green_pct:.1f}%")
    print(f"   Day 0 was RED:   {red_pct:.1f}%")
    
    if red_pct > 60:
        print(f"   ✅ SIGNAL: Red Day 0 often precedes runs ({red_pct:.0f}%)")
    elif green_pct > 60:
        print(f"   ✅ SIGNAL: Green Day 0 often precedes runs ({green_pct:.0f}%)")
    else:
        print(f"   ⚠️ MIXED: No clear color signal")
    
    # PATTERN 2: Day 0 Volume
    print("\n📊 PATTERN 2: Day 0 Volume")
    print("-" * 50)
    avg_vol = day0_df['day0_vol'].mean()
    high_vol_pct = (day0_df['day0_vol'] > 1.5).sum() / len(day0_df) * 100
    print(f"   Average Day 0 volume ratio: {avg_vol:.2f}x")
    print(f"   Day 0 with 1.5x+ volume: {high_vol_pct:.1f}%")
    
    if avg_vol > 1.3:
        print(f"   ✅ SIGNAL: Volume spikes on Day 0 ({avg_vol:.2f}x avg)")
    else:
        print(f"   ⚠️ WEAK: Volume not significantly elevated on Day 0")
    
    # PATTERN 3: Day -1 and Day -2 (Consolidation)
    print("\n📉 PATTERN 3: Pre-Day 0 Consolidation (Day -2, Day -1)")
    print("-" * 50)
    
    # Check for "rest then run" pattern (2 red days before Day 1)
    rest_pattern = ((~day0_df['day_m2_green']) & (~day0_df['day_m1_green'])).sum() / len(day0_df) * 100
    one_red = ((~day0_df['day_m1_green']) | (~day0_df['day_m2_green'])).sum() / len(day0_df) * 100
    
    print(f"   2 red days before Day 0: {rest_pattern:.1f}%")
    print(f"   At least 1 red day before Day 0: {one_red:.1f}%")
    
    if rest_pattern > 40:
        print(f"   ✅ SIGNAL: 'Rest then run' pattern detected ({rest_pattern:.0f}%)")
    
    # PATTERN 4: Day 1 Gap
    print("\n🚀 PATTERN 4: Day 1 Gap Up")
    print("-" * 50)
    avg_gap = day0_df['day1_gap'].mean()
    gap_up_pct = (day0_df['day1_gap'] > 2).sum() / len(day0_df) * 100
    print(f"   Average Day 1 gap: {avg_gap:+.1f}%")
    print(f"   Day 1 with 2%+ gap: {gap_up_pct:.1f}%")
    
    if gap_up_pct > 50:
        print(f"   ✅ SIGNAL: Most Day 1's gap up ({gap_up_pct:.0f}%)")
        print(f"   🎯 ACTIONABLE: Watch pre-market for 2%+ gaps after rest days")
    
    # PATTERN 5: Day 1 Volume
    print("\n📈 PATTERN 5: Day 1 Volume Confirmation")
    print("-" * 50)
    avg_day1_vol = day0_df['day1_vol'].mean()
    high_day1_vol = (day0_df['day1_vol'] > 2).sum() / len(day0_df) * 100
    print(f"   Average Day 1 volume ratio: {avg_day1_vol:.2f}x")
    print(f"   Day 1 with 2x+ volume: {high_day1_vol:.1f}%")
    
    if avg_day1_vol > 2:
        print(f"   ✅ CONFIRMED: Day 1's have volume confirmation ({avg_day1_vol:.1f}x avg)")


📊 DAY 0 PATTERN ANALYSIS

Sample size: 68 Day 1's analyzed


📈 PATTERN 1: Day 0 Color (Green vs Red before Day 1)
--------------------------------------------------
   Day 0 was GREEN: 0.0%
   Day 0 was RED:   100.0%
   ✅ SIGNAL: Red Day 0 often precedes runs (100%)

📊 PATTERN 2: Day 0 Volume
--------------------------------------------------
   Average Day 0 volume ratio: 1.09x
   Day 0 with 1.5x+ volume: 8.8%
   ⚠️ WEAK: Volume not significantly elevated on Day 0

📉 PATTERN 3: Pre-Day 0 Consolidation (Day -2, Day -1)
--------------------------------------------------
   2 red days before Day 0: 22.1%
   At least 1 red day before Day 0: 75.0%

🚀 PATTERN 4: Day 1 Gap Up
--------------------------------------------------
   Average Day 1 gap: +2.6%
   Day 1 with 2%+ gap: 42.6%

📈 PATTERN 5: Day 1 Volume Confirmation
--------------------------------------------------
   Average Day 1 volume ratio: 1.11x
   Day 1 with 2x+ volume: 5.9%


In [7]:
# COMBINED DAY 0 SIGNAL TEST
if all_day0_data:
    print("\n" + "="*80)
    print("🎯 COMBINED SIGNAL TEST: Does Multi-Factor Day 0 Work?")
    print("="*80)
    
    # Define Day 0 signal:
    # - Day -1 or Day -2 was red (rest)
    # - Day 0 has elevated volume (>1.2x)
    # - Day 0 price held (not down >3%)
    
    day0_df['signal'] = (
        ((~day0_df['day_m1_green']) | (~day0_df['day_m2_green'])) &  # Rest period
        (day0_df['day0_vol'] > 1.0) &  # Some volume
        (day0_df['day0_pct'] > -5)  # Didn't crash
    )
    
    signal_hit_rate = day0_df['signal'].sum() / len(day0_df) * 100
    
    print(f"\nDay 0 Signal Definition:")
    print(f"   - Recent rest (red day in prior 2 days)")
    print(f"   - Volume present (>1.0x avg)")
    print(f"   - Price stable (not down >5%)")
    print(f"\nSignal present before {signal_hit_rate:.0f}% of Day 1's")
    
    if signal_hit_rate >= 70:
        print(f"\n✅ STRONG SIGNAL: Day 0 pattern is predictive ({signal_hit_rate:.0f}%)")
    elif signal_hit_rate >= 50:
        print(f"\n⚠️ MODERATE SIGNAL: Day 0 pattern has some predictive value ({signal_hit_rate:.0f}%)")
    else:
        print(f"\n❌ WEAK SIGNAL: Day 0 pattern not reliable ({signal_hit_rate:.0f}%)")
    
    # Show examples
    print(f"\n📋 SAMPLE Day 0 → Day 1 Transitions:")
    print("-" * 70)
    
    for i, d in enumerate(all_day0_data[:10]):
        day0_color = "🟢" if d['day0']['green'] else "🔴"
        print(f"   {d['ticker']:6} | Day 0: {day0_color} {d['day0']['pct_change']:+5.1f}%, "
              f"Vol {d['day0']['vol_ratio']:.1f}x | "
              f"Day 1: +{d['day1']['pct_change']:.1f}%, Run: {d['run_length']}d, "
              f"Total: +{d['total_gain']:.0f}%")


🎯 COMBINED SIGNAL TEST: Does Multi-Factor Day 0 Work?

Day 0 Signal Definition:
   - Recent rest (red day in prior 2 days)
   - Volume present (>1.0x avg)
   - Price stable (not down >5%)

Signal present before 9% of Day 1's

❌ WEAK SIGNAL: Day 0 pattern not reliable (9%)

📋 SAMPLE Day 0 → Day 1 Transitions:
----------------------------------------------------------------------
   SIDU   | Day 0: 🔴  -1.7%, Vol 0.4x | Day 1: +0.9%, Run: 4d, Total: +13%
   SIDU   | Day 0: 🔴 -28.7%, Vol 7.1x | Day 1: +10.5%, Run: 3d, Total: +15%
   SIDU   | Day 0: 🔴  -0.9%, Vol 0.6x | Day 1: +5.5%, Run: 3d, Total: +30%
   SIDU   | Day 0: 🔴  -9.3%, Vol 1.0x | Day 1: +1.3%, Run: 3d, Total: +27%
   SIDU   | Day 0: 🔴  -8.1%, Vol 0.6x | Day 1: +3.2%, Run: 4d, Total: +59%
   SIDU   | Day 0: 🔴 -12.3%, Vol 4.8x | Day 1: +33.2%, Run: 4d, Total: +124%
   ASTS   | Day 0: 🔴  -0.2%, Vol 0.4x | Day 1: +4.6%, Run: 3d, Total: +12%
   ASTS   | Day 0: 🔴  -9.5%, Vol 2.3x | Day 1: +1.8%, Run: 4d, Total: +11%
   ASTS   | Day

In [8]:
# FALSE POSITIVE TEST
if all_day0_data:
    print("\n" + "="*80)
    print("⚠️ FALSE POSITIVE TEST: How often does signal appear WITHOUT a run?")
    print("="*80)
    print("\nThis is CRITICAL - if signal fires constantly, it's useless.")
    print("\nTesting: Count all days with 'Day 0 signal' vs actual Day 1's...\n")
    
    # For a sample ticker, count how many times signal appears vs actual Day 1's
    test_ticker = 'SIDU'
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    
    stock = yf.Ticker(test_ticker)
    hist = stock.history(start=start_date, end=end_date)
    
    hist['green'] = hist['Close'] > hist['Close'].shift(1)
    hist['vol_ratio'] = hist['Volume'] / hist['Volume'].rolling(20).mean()
    hist['pct_change'] = hist['Close'].pct_change() * 100
    
    # Day 0 signal: recent red + volume + stable price
    hist['recent_red'] = (~hist['green'].shift(1)) | (~hist['green'].shift(2))
    hist['signal'] = (
        hist['recent_red'] &
        (hist['vol_ratio'] > 1.0) &
        (hist['pct_change'] > -5)
    )
    
    # Count actual Day 1's (simplified: 3+ consecutive green days starting)
    hist['next_3_green'] = (
        hist['green'].shift(-1) &
        hist['green'].shift(-2) &
        hist['green'].shift(-3)
    )
    
    total_signals = hist['signal'].sum()
    signals_with_run = (hist['signal'] & hist['next_3_green']).sum()
    false_positives = total_signals - signals_with_run
    false_positive_rate = false_positives / total_signals * 100 if total_signals > 0 else 0
    
    print(f"Test ticker: {test_ticker}")
    print(f"Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    print(f"\nTotal 'Day 0 signals' detected: {total_signals}")
    print(f"Signals followed by 3+ day run: {signals_with_run}")
    print(f"False positives (signal but no run): {false_positives}")
    print(f"\nFalse positive rate: {false_positive_rate:.1f}%")
    
    if false_positive_rate < 30:
        print(f"\n✅ LOW FALSE POSITIVES: Signal is tradeable ({100-false_positive_rate:.0f}% accuracy)")
    elif false_positive_rate < 50:
        print(f"\n⚠️ MODERATE FALSE POSITIVES: Need additional filters ({100-false_positive_rate:.0f}% accuracy)")
    else:
        print(f"\n❌ HIGH FALSE POSITIVES: Signal fires too often ({100-false_positive_rate:.0f}% accuracy)")
        print(f"   Need stricter criteria or additional confirmation")


⚠️ FALSE POSITIVE TEST: How often does signal appear WITHOUT a run?

This is CRITICAL - if signal fires constantly, it's useless.

Testing: Count all days with 'Day 0 signal' vs actual Day 1's...



TypeError: bad operand type for unary ~: 'float'

In [9]:
# THESIS VERDICT
print("\n" + "="*80)
print("🎯 THESIS VERDICT: DAY 0 DETECTION")
print("="*80)

print("\n📊 SUMMARY OF FINDINGS:")
print("-" * 40)

findings = []

if all_day0_data:
    # Compile findings
    if red_pct > 55:
        findings.append(f"✅ Day 0 often RED ({red_pct:.0f}%) - 'rest before run'")
    if avg_vol > 1.2:
        findings.append(f"✅ Day 0 has elevated volume ({avg_vol:.1f}x)")
    if rest_pattern > 40:
        findings.append(f"✅ Rest pattern detected ({rest_pattern:.0f}% have 2 red days before)")
    if gap_up_pct > 50:
        findings.append(f"✅ Day 1 gaps up ({gap_up_pct:.0f}%) - pre-market signal")
    if avg_day1_vol > 1.5:
        findings.append(f"✅ Day 1 has volume surge ({avg_day1_vol:.1f}x)")
    
    for f in findings:
        print(f"   {f}")
    
    print(f"\n📋 ACTIONABLE DAY 0 CHECKLIST:")
    print("-" * 40)
    print("   To catch Day 1 BEFORE it happens, look for:")
    print("   □ Serial runner that just had 1-2 red days (rest period)")
    print("   □ Volume picking up on flat/slightly red day")
    print("   □ Pre-market shows 2%+ gap up")
    print("   □ Volume surge at open confirms Day 1")
    print("\n   ⏰ TIMING: Check pre-market 6:30-9:30 AM on rest day ends")

if len(findings) >= 3:
    print(f"\n✅ THESIS VALIDATED: Day 0 signals ARE predictive")
    print(f"   Found {len(findings)} reliable signals")
elif len(findings) >= 1:
    print(f"\n⚠️ THESIS PARTIALLY VALIDATED: Some Day 0 signals exist")
    print(f"   Found {len(findings)} signals - use in combination")
else:
    print(f"\n❌ THESIS KILLED: No reliable Day 0 patterns found")

print("\n" + "="*80)
print("NEXT: Run Notebook 3 (Rotation Chain) to test institutional flow")
print("="*80)


🎯 THESIS VERDICT: DAY 0 DETECTION

📊 SUMMARY OF FINDINGS:
----------------------------------------
   ✅ Day 0 often RED (100%) - 'rest before run'

📋 ACTIONABLE DAY 0 CHECKLIST:
----------------------------------------
   To catch Day 1 BEFORE it happens, look for:
   □ Serial runner that just had 1-2 red days (rest period)
   □ Volume picking up on flat/slightly red day
   □ Pre-market shows 2%+ gap up
   □ Volume surge at open confirms Day 1

   ⏰ TIMING: Check pre-market 6:30-9:30 AM on rest day ends

⚠️ THESIS PARTIALLY VALIDATED: Some Day 0 signals exist
   Found 1 signals - use in combination

NEXT: Run Notebook 3 (Rotation Chain) to test institutional flow
